In [44]:
from pipeline import ClaimifyPipeline
from llm_client import LLMClient
import polars as pl
import json
import os

classification_data_path = os.path.join(os.getcwd(), "../Data-Analysis-Project-28/data_for_git/covid_classifications.jsonl")

# load with utf-8 encoding
articles = []
with open(classification_data_path, "r", encoding="utf-8") as f:
    for line in f:
        articles.append(json.loads(line))

article_data_path = os.path.join(os.getcwd(), "../Data-Analysis-Project-28/data/article_texts_with_metadata.parquet")

article_data = pl.read_parquet(article_data_path)

print(articles[0]["result"])

kategorier = set([article["result"]["huvudkategori"] for article in articles])

kategori = "Svenska strategin"
selected_articles = []
for article in articles:
    if article["result"]["huvudkategori"] == kategori:
        selected_articles.append(article)

print(len(selected_articles))

combined_data = []
for article in selected_articles:
    combined = {"classification": article["result"], "article": article_data.filter(pl.col("id") == article["id"]).to_dicts()[0]}
    combined_data.append(combined)

print(combined_data[1]["article"]["publication"])
print(combined_data[1]["article"]["body_text"])

{'resonemang': 'Artikeln rapporterar om andelen personer med antikroppar mot covid-19 i olika svenska städer, vilket handlar om befolkningens immunitet mot viruset.', 'kategorier': ['Immunitet'], 'huvudkategori': 'Immunitet', 'källor': None, 'personer': ['Henrik Forsberg'], 'länder': ['Sverige']}
322
aftonbladet
Nya boken ”Flocken” avslöjar spelet bakom den så kallade svenska strategin.

Genom att gå igenom en mängd mejl har författaren Johan Anderberg pusslat ihop födelsen av statsepidemiolog Anders Tegnells covid-19-strategi. Svenska Dagbladet har fått tag på ett bearbetat, ännu opublicerat utdrag ur boken som beskriver i detalj spelet bakom det som skulle bli Sveriges förmodligen mest debatterade vägval året 2020.
Den 6 mars fick Anders Tegnell ett varnande mejl från matematikern Tom Britton som räknat på R-värdet för viruset. Där framförde han sin åsikt att  Folkhälsomyndigheten var för passiv: ”Med så många smittade som nu finns i Sverige så kommer smittspridning att ske. Vårt sam

In [45]:
import threading
os.environ["OPENAI_API_KEY"] = ""
model = "openai/gpt-oss-120b"
llm_client = LLMClient(model)

pipeline = ClaimifyPipeline(llm_client, question="Vad uttrycker texten om den svenska strategin för att hantera Coronapandemin?")
results = {}
result_lock = threading.Lock()


def run_pipeline(sample, result, result_lock):
    article = sample["article"]
    prompt = article["title"] + "\n" + article["actual_lead_text"] + "\n" + article["body_text"]
    result = pipeline.run(prompt)
    with result_lock:
        results[article["id"]] = result


threads = []
for i, sample in enumerate(combined_data[1:2]):
    thread = threading.Thread(target=run_pipeline, args=(sample, results, result_lock))
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

# save result to json
notebook_dir = os.getcwd()
dir_path = os.path.join(notebook_dir, "claims_results")
print(dir_path)
if not os.path.exists(dir_path):
    os.makedirs(dir_path)
# find the current highest run number
if not os.path.exists(os.path.join(dir_path, "current_run.json")):
    with open(os.path.join(dir_path, "current_run.json"), "w") as f:
        json.dump({model: 0}, f)
with open(os.path.join(dir_path, "current_run.json"), "r") as f:
    runs_dict = json.load(f)
    if runs_dict.get(model) is None:
        run_number = 0
        runs_dict[model] = 0
    else:
        run_number = runs_dict[model]



file_path = os.path.join(dir_path, f"{model.split('/')[-1]}_{run_number}.json")
with open(file_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False)

runs_dict[model] = run_number + 1
with open(os.path.join(dir_path, "current_run.json"), "w") as f:
    json.dump(runs_dict, f)


Parsed response: språk='svenska' mening='Till sist fick de tre männen komma till myndigheten.' tankeprocess='Analys av textens innebörd' slutlig_bedömning='Innehåller ett specifikt och verifierbart påstående' mening_med_endast_verifierbar_info=None
Parsed response: språk='svenska' mening='Om Tegnell fick en liten lucka kunde de vara där inom 30 minuter, enligt SvD.' tankeprocess='Den här meningen ger en konkret bild av hur snabbt experter kunde nå Folkhälsomyndigheten när de såg ett akut behov av att varna Teknell. Den används i texten för att understryka att myndigheten var tillgänglig men ändå valde att inte agera på de tidiga larmen.' slutlig_bedömning='Innehåller ett specifikt och verifierbart påstående' mening_med_endast_verifierbar_info='Det är Korrekt.'
Parsed response: språk='sv' mening='I dag, snart åtta månader senare, har drygt 6\u202f000 svenskar avlidit i covid‑19.' tankeprocess='I analysen tolkar jag texten som en reflektion över hur den svenska pandemistrategin uppfattas

In [46]:
from sklearn.metrics.pairwise import cosine_similarity
claims = []
for text_claims in results.values():
    for claim in text_claims:
        claims.append(claim)
        print(claim)

from openai import OpenAI
Client = OpenAI(base_url="http://localhost:4000/v1", api_key="EMPTY")

def get_embeddings(prompts):
    response = Client.embeddings.create(
        model="Qwen/Qwen3-Embedding-8B",
        input=prompts
    )
    return [item.embedding for item in response.data]

print("Embedding claims...")
embeddings = get_embeddings(claims)
print("Done embedding claims")

Det framkommer att Folkhälsomyndigheten betraktades som alltför passiv och att viktiga medicinska röster tvingades insistera på att träffa Anders Tegnell för att påminna om den akuta risken med viruset.
Det indikerar att den officiella covid‑strategin i Sverige saknade en snabb, proaktiv hållning och därför behövde kompletteras av interna varningar från framstående infektions‑ och smittskyddsexperter.
Den svenska strategin har skildrats som en medveten men bristfällig policy, där val av flockimmunitet valdes framför andra möjliga åtgärder, och där avsaknaden av en klar, styrande hand sades leda till ett onödigt stort antal dödsfall.
Sveriges covid‑19‑strategi var i praktiken ett medvetet val av den tredje möjliga vägen – att låta viruset spridas för att nå herd immunity – en strategi som författaren beskriver som både uppgiven och huvudlös.
Strategin sägs ha satts i verket trots varningar om höga dödstal och presenterar en ”huvudlös” linje med lågt R‑värde‑klimat.
Myndigheten var otymp

In [47]:
embedding_groups = []
for embedding, claim in zip(embeddings, claims):
    if len(embedding_groups) == 0:
        embedding_groups.append({"representative": {"claim": claim, "embedding": embedding}, "members": [claim]})
    else:
        most_similar_group = max(embedding_groups, key=lambda group: cosine_similarity([embedding], [group["representative"]["embedding"]])[0])
        if most_similar_group is not None and cosine_similarity([embedding], [most_similar_group["representative"]["embedding"]])[0] > 0.65:
            most_similar_group["members"].append(claim)
        else:
            embedding_groups.append({"representative": {"claim": claim, "embedding": embedding}, "members": [claim]})


for group in embedding_groups:
    print("Representative: " + group["representative"]["claim"])
    print(f"Members: {len(group['members'])}")
    # for member in group["members"]:
    #     print(member)
    # print("---")


Representative: Det framkommer att Folkhälsomyndigheten betraktades som alltför passiv och att viktiga medicinska röster tvingades insistera på att träffa Anders Tegnell för att påminna om den akuta risken med viruset.
Members: 1
Representative: Det indikerar att den officiella covid‑strategin i Sverige saknade en snabb, proaktiv hållning och därför behövde kompletteras av interna varningar från framstående infektions‑ och smittskyddsexperter.
Members: 7
Representative: Sveriges covid‑19‑strategi var i praktiken ett medvetet val av den tredje möjliga vägen – att låta viruset spridas för att nå herd immunity – en strategi som författaren beskriver som både uppgiven och huvudlös.
Members: 13
Representative: Strategin sägs ha satts i verket trots varningar om höga dödstal och presenterar en ”huvudlös” linje med lågt R‑värde‑klimat.
Members: 1
Representative: Myndigheten var otymplig i att omsätta expertvarningarna i konkreta, tidiga restriktioner.
Members: 1
Representative: Strategin port

In [ ]:
from pydantic import BaseModel
from typing import Literal
class RecognizingContext(BaseModel):
    text_topic: str
    author_identity: str
    target_audience: str
    socio_cultural_background: str

class AnalyzingMainIdea(BaseModel):
    core_viewpoint: str
    main_intention: str

class EmotionalAnalysis(BaseModel):
    language_expression: str
    rhetorical_strategy: str
    tone: str
    emotion: str

class StanceReinforcement(BaseModel):
    favor: str
    against: str
    none: str

class LogicalInference(BaseModel):
    reasoning_summary: str

class StanceDetermination(BaseModel):
    stance_label: Literal["FAVOR", "AGAINST", "NONE"]

class ChainOfStanceOutput(BaseModel):
    recognizing_context: RecognizingContext
    analyzing_main_idea: AnalyzingMainIdea
    emotional_analysis: EmotionalAnalysis
    stance_reinforcement: StanceReinforcement
    logical_inference: LogicalInference
    stance_determination: StanceDetermination

GUIDED_SCHEMA = ChainOfStanceOutput.model_json_schema()

In [ ]:
FEW_SHOT_EXAMPLES = [
    {
        "role": "system",
        "content": (
            "Du är en expertmodell för ställningsdetektion (stance detection). "
            "Givet ett **mål** och en **text** måste du dra slutsatsen om författarens ställning (stance) "
            "till målet (FÖR/FAVOR, EMOT/AGAINST, eller INGEN/NONE).\n\n"
            "Du svarar alltid på svenska med ett enda JSON-objekt som matchar detta schema:\n"
            "{\n"
            '  "recognizing_context": {\n'
            '    "text_topic": str,\n'
            '    "author_identity": str,\n'
            '    "target_audience": str,\n'
            '    "socio_cultural_background": str\n'
            "  },\n"
            '  "analyzing_main_idea": {\n'
            '    "core_viewpoint": str,\n'
            '    "main_intention": str\n'
            "  },\n"
            '  "emotional_analysis": {\n'
            '    "language_expression": str,\n'
            '    "rhetorical_strategy": str,\n'
            '    "tone": str,\n'
            '    "emotion": str\n'
            "  },\n"
            '  "stance_reinforcement": {\n'
            '    "favor": str,\n'
            '    "against": str,\n'
            '    "none": str\n'
            "  },\n"
            '  "logical_inference": {\n'
            '    "reasoning_summary": str\n'
            "  },\n"
            '  "stance_determination": {\n'
            '    "stance_label": "FAVOR" | "AGAINST" | "NONE"\n'
            "  }\n"
            "}\n\n"
            "Var **kortfattad** men **specifik**. Lägg inte till extra fält. "
            "Linda inte in JSON-objektet i backticks."
        ),
    },

    # ===== EXEMPEL 1: EMOT (AGAINST) =====
    {
        "role": "user",
        "content": (
            "Mål: Hillary Clinton\n\n"
            "Text: Jag älskar doften av Hillary på morgonen. "
            "Den luktar som Republikansk seger."
        ),
    },
    {
        "role": "assistant",
        "content": """
{
  "recognizing_context": {
    "text_topic": "Texten refererar till Hillary Clinton och en kommande politisk tävling.",
    "author_identity": "En politiskt engagerad användare av sociala medier som sannolikt lutar åt det Republikanska partiet.",
    "target_audience": "Personer som följer amerikansk politik, särskilt konservativa eller Republikanskt lutande läsare.",
    "socio_cultural_background": "Amerikansk valperiod där Hillary Clinton associeras med det Demokratiska partiet och Republikaner framställer henne som en motståndare att besegra."
  },
  "analyzing_main_idea": {
    "core_viewpoint": "Författaren hånar Hillary Clinton och antyder att hennes närvaro leder till en Republikansk seger.",
    "main_intention": "Att förringa Hillary Clinton samtidigt som man uttrycker entusiasm för att Republikanerna ska vinna."
  },
  "emotional_analysis": {
    "language_expression": "Frasen \"Jag älskar doften av\" används sarkastiskt, vilket antyder glädje inte i Hillary själv utan i hennes nederlag.",
    "rhetorical_strategy": "Parodi på ett känt filmcitat, vridet för att förlöjliga Hillary Clinton och fira Republikansk framgång.",
    "tone": "Sarkastisk och hånfull.",
    "emotion": "Skadeglädje och fientlighet mot Hillary Clinton, spänning över Republikanerna."
  },
  "stance_reinforcement": {
    "favor": "Det finns inget beröm eller stöd för Hillary Clinton.",
    "against": "Texten hånar henne och ramar in henne som en symbol för Republikansk seger, vilket antyder att hon är något som ska besegras.",
    "none": "Meddelandet är känslomässigt laddat och helt klart inte neutralt."
  },
  "logical_inference": {
    "reasoning_summary": "Eftersom författaren använder sarkasm och hån riktat mot Hillary Clinton, samtidigt som han ramar in henne som en väg till Republikansk seger, är ställningen tydligt emot henne.",
  },
  "stance_determination": {
    "stance_label": "AGAINST"
  }
}
""".strip()
    },

    # ===== EXEMPEL 2: FÖR (FAVOR) =====
    {
        "role": "user",
        "content": (
            "Mål: Subventioner för förnybar energi\n\n"
            "Text: Att subventionera ren energi är en av de smartaste investeringarna vi kan göra. "
            "Det skapar jobb, minskar föroreningar och håller oss konkurrenskraftiga på lång sikt."
        ),
    },
    {
        "role": "assistant",
        "content": """
{
  "recognizing_context": {
    "text_topic": "Statliga subventioner för rena eller förnybara energiprojekt.",
    "author_identity": "En policymedveten individ som stöder miljömässiga och ekonomiska fördelar med ren energi.",
    "target_audience": "Läsare intresserade av offentlig politik, klimatfrågor och ekonomisk tillväxt.",
    "socio_cultural_background": "Pågående offentliga debatter om klimatförändringar, statliga utgifter och grön industripolitik."
  },
  "analyzing_main_idea": {
    "core_viewpoint": "Att subventionera ren energi är fördelaktigt och strategiskt klokt.",
    "main_intention": "Att förespråka förnybara energisubventioner genom att lista positiva resultat.",
  },
  "emotional_analysis": {
    "language_expression": "Fraser som \"smartaste investeringarna\" och \"håller oss konkurrenskraftiga\" förmedlar starkt stöd.",
    "rhetorical_strategy": "Appell till praktiska fördelar: jobb, minskade föroreningar och ekonomisk konkurrenskraft.",
    "tone": "Optimistisk och gillande.",
    "emotion": "Hoppfullhet och tillförsikt om den positiva effekten av sådana subventioner."
  },
  "stance_reinforcement": {
    "favor": "Texten hävdar uttryckligen att subventioner är en smart investering och listar fördelar.",
    "against": "Det finns ingen kritik mot subventioner eller omnämnande av kostnader som ett problem.",
    "none": "Meddelandet är tydligt åsiktstungt snarare än neutralt."
  },
  "logical_inference": {
    "reasoning_summary": "Genom att beskriva subventioner som smarta och fördelaktiga i flera dimensioner, förmedlar författaren tydligt stöd för förnybara energisubventioner.",
  },
  "stance_determination": {
    "stance_label": "FAVOR"
  }
}
""".strip()
    }
]

In [ ]:
# build prompt with system prompt, few shot examples and user query
def build_prompt(few_shot_examples: list[str], text: str, target: str) -> str:
    # for each example in few_shot_examples, add it as message history openAI style with role and content to the prompt
    messages = []
    for example in few_shot_examples:
        messages.append({"role": example['role'], "content": example['content']})

    # add user query at the end
    messages.append({"role": "user", "content": f"Target: {target}\n\nText: {text}"})
    return messages

In [ ]:
client = OpenAI(
    api_key="",
    base_url="http://localhost:8000/v1",
    max_retries=0,
    timeout=60
)

def detect_stance_chain(text: str, target: str) -> ChainOfStanceOutput:
    messages = build_prompt(FEW_SHOT_EXAMPLES, text, target)
    model_name = "meta-llama/llama-3.3-70b-instruct:free"
    # `response_format` uses the Pydantic model as a guided schema
    # response = client.responses.create(
    #     model="gpt-4.1-mini",  # or another model that supports structured output
    #     input=prompt,
    #     response_format=ChainOfStanceOutput,  # <- Pydantic model here
    # )

    response = client.chat.completions.create(
            messages=messages,
            model=model_name,
            temperature=0.3,
            max_completion_tokens=1024,
            extra_body={
                    "response_format": {
                        "type": "json_schema",
                        "json_schema": {
                            "name": "ChainOfStanceOutput",
                            "strict": True,
                            "schema": GUIDED_SCHEMA,
                        },
                    }
                },
            )


    # Parsed result is already a Pydantic model instance
    raw_content = response.choices[0].message.content
    return ChainOfStanceOutput.model_validate_json(raw_content)

In [ ]:
for group in embedding_groups:
    claim = group["representative"]["claim"]
    print(detect_stance_chain(claim, "Svenska strategin för Coronapandemin"))
    print("---")


In [35]:
from pydantic import BaseModel
from openai import OpenAI
from typing import List
class Claim(BaseModel):
    reasoning: List[str]
    claim: str
    evidence: str
    validity_of_claim: str

system_prompt = """
You are a helpful assistant that extracts claims from a text. Let reasoning capture your
full thought process and the later fields capture your final answer.
"""

user_prompt = """
Paris is the worst city in the world. When I went there I lost my wallet.
"""

response_model = Claim

client = OpenAI(base_url="http://localhost:8000/v1", api_key="")

response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": user_prompt}],
                extra_body={
                    "response_format": {
                        "type": "json_schema",
                        "json_schema": {
                            "name": "Category",
                            "strict": True,
                            "schema": Claim.model_json_schema(),
                        },
                    }
                },
            )
print(response)

ChatCompletion(id='chatcmpl-8985694e666ccee4', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"reasoning": ["The user is expressing a personal negative opinion about a city, which is allowed under the policy as it is not targeting a protected class. The user also shared an experience of losing a wallet, which we can respond to empathetically and offer helpful advice."], "claim": "The user is sharing a personal negative opinion about a city and a personal experience of losing a wallet. This is allowed content. We can respond with empathy, ask follow‑up questions, and provide practical suggestions for what to do after losing a wallet while traveling." , "evidence": "" ,"validity_of_claim": "The claim is accurate."}', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning='The user made a negative statement about Paris. This is political or potentially hateful content? It\'s an opini

In [36]:
print(response.choices[0].message.content)
# print(response.choices[0].message.reasoning)

{"reasoning": ["The user is expressing a personal negative opinion about a city, which is allowed under the policy as it is not targeting a protected class. The user also shared an experience of losing a wallet, which we can respond to empathetically and offer helpful advice."], "claim": "The user is sharing a personal negative opinion about a city and a personal experience of losing a wallet. This is allowed content. We can respond with empathy, ask follow‑up questions, and provide practical suggestions for what to do after losing a wallet while traveling." , "evidence": "" ,"validity_of_claim": "The claim is accurate."}
